In [1]:
import numpy as np 
import os
import tensorflow as tf
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras import Sequential
from tensorflow.keras import layers

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
tf.compat.v1.disable_eager_execution()

from keras.utils.np_utils import to_categorical
from keras import metrics
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Dense, Dropout, Flatten, Activation,Concatenate
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.metrics import Recall
from keras.metrics import Precision
from tensorflow.keras.utils import plot_model
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import ResNet50


import cv2
from tqdm import tqdm

from sklearn.metrics import classification_report
from sklearn.utils import shuffle 
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight


import math
import random


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def load_data():
    
    #datasets = ["C:/Users/dodi/Desktop/NEURAL PROJECT/TRAIN_DATA150x150" , "C:/Users/dodi/Desktop/NEURAL PROJECT/TEST_DATA150x150"]
    datasets = ["/content/drive/MyDrive/Neural Project/TRAIN_DATA150x150" , "/content/drive/MyDrive/Neural Project/TEST_DATA150x150"]
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [4]:
class_names = ["damaged_infrastructure", "damaged_nature", "fires", "flood", "human_damage", "non_damage"]
class_names_label = {class_name: i for i, class_name in enumerate(class_names)}
num_classes = len(class_names)

## Test to check that our classes are correct for classifcation:
print(num_classes)
print(class_names)

IMAGE_SIZE = (150,150)

6
['damaged_infrastructure', 'damaged_nature', 'fires', 'flood', 'human_damage', 'non_damage']


In [5]:
def generate_actual_predicted(model, X_test, Y_test):
    # Get the predictions
    Y_pred = model.predict(X_test)

    # Create a list of predictions

    #Converting predictions to label
    predicted = list()
    for i in range(len(Y_pred)):
        predicted.append(np.argmax(Y_pred[i]))

    #Converting one hot encoded test label back to label
    actual = list()
    for i in range(len(Y_test)):
        actual.append(np.argmax(Y_test[i]))

    return actual, predicted

In [6]:
(train_images, train_labels), (test_images, test_labels)  = load_data()
print("Done")

Loading /content/drive/MyDrive/Neural Project/TRAIN_DATA150x150


100%|██████████| 192/192 [01:17<00:00,  2.48it/s]


Loading /content/drive/MyDrive/Neural Project/TEST_DATA150x150


100%|██████████| 69/69 [00:28<00:00,  2.40it/s]

Done


In [29]:
# What if we wish to do K-fold 

def create_model():
  model = tf.keras.Sequential()

  model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)))
  model.add(layers.ZeroPadding2D( padding = 1 ))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
  model.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
  model.add(layers.ZeroPadding2D( padding = 1 ))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation=tf.nn.relu))

  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(6, activation=tf.nn.softmax))
 
  model.compile(
                optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'],
                )
  return model


In [30]:
print(train_labels)
train_labels = to_categorical(train_labels)
print(train_labels)
print(train_labels.shape)

[1 1 1 ... 4 4 4]
[[0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]]
(4715, 6)


In [ ]:
n_split=10
 
# We will split the the data n_split times
# and create and for the model with the said
# parameters

results = list()
for train_index,test_index in KFold(n_split).split(train_images): 
  # use the index to generate training an testing sets
  x_train,x_test=train_images[train_index],train_images[test_index]
  y_train,y_test=train_labels[train_index],train_labels[test_index]
  
  # create and fit the model
  model=create_model()
  model.fit(x_train, y_train,epochs=2)
  
  # print and show generic metrics available
  scores = model.evaluate(x_test,y_test)
 
  # print the model metrics
  for i,names in zip(np.arange(0,len(model.metrics_names)),model.metrics_names):
      print(model.metrics_names[i],'=',scores[i])
  # calculate and print more metrics
  actual, predicted = generate_actual_predicted(model, x_test, y_test)
  print("precision ",precision_score(actual, predicted, average='macro'))
  print("recall ", recall_score(actual, predicted, average='macro'))
  print("F1-Score ", f1_score(actual, predicted, average='macro'))
  results.append([precision_score(actual, predicted, average='macro'),
                  recall_score(actual, predicted, average='macro'), 
                  f1_score(actual, predicted, average='macro') ])

Train on 4243 samples
Epoch 1/2
4243/4243 [==============================] - 575s 135ms/sample - loss: 8.8816 - accuracy: 0.4485
Epoch 2/2
4243/4243 [==============================] - 571s 135ms/sample - loss: 7.0885 - accuracy: 0.5602


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


loss = 16.11809539794922
accuracy = 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision  0.0
recall  0.0
F1-Score  0.0
Train on 4243 samples
Epoch 1/2
4243/4243 [==============================] - 573s 135ms/sample - loss: 10.7927 - accuracy: 0.3302
Epoch 2/2
4243/4243 [==============================] - 574s 135ms/sample - loss: 8.7105 - accuracy: 0.4596
loss = 16.11809539794922
accuracy = 0.0
precision  0.0
recall  0.0
F1-Score  0.0
Train on 4243 samples
Epoch 1/2
4243/4243 [==============================] - 574s 135ms/sample - loss: 10.6953 - accuracy: 0.3361
Epoch 2/2
4243/4243 [==============================] - 573s 135ms/sample - loss: 7.0999 - accuracy: 0.5595
loss = 16.11809539794922
accuracy = 0.0
precision  0.0
recall  0.0
F1-Score  0.0
Train on 4243 samples
Epoch 1/2
4243/4243 [==============================] - 571s 135ms/sample - loss: 7.2582 - accuracy: 0.5494
Epoch 2/2
4243/4243 [==============================] - 577s 136ms/sample - loss: 7.1911 - accuracy: 0.5539
loss = 15.332679737422426
accuracy = 0.048728812
precision  0.024364406779661018
recall

In [ ]:
# Plot the curves for precision
res = np.array(results)
#print(res)

prec = res[:,0]
print(prec)
recl = res[:,1]
print(recl)

import matplotlib.pyplot as plt
plt.scatter(prec, recl)
plt.xlabel('precision')
plt.ylabel('recall')
plt.title('K-Fold') 
plt.show()

print('min precision:', prec.min())
print('min recall:', recl.min())
print('max precision:', prec.max())
print('max recall:', recl.max())
print('average precision:', prec.mean(), 'sd=', prec.std())
print('average recall:', recl.mean(),'sd=', recl.std())

In [ ]:
# show f1-score for k-fold
res = np.array(results)
f1_score = res[:,2]
plt.hist(x=f1_score, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.title('K-Fold F1-Score') 
plt.ylabel('Frequency') 
plt.xlabel('F1-Score') 
plt.show()
print('F1-Score:', f1_score.mean(), 'sd=', f1_score.std())

res_model1 = res
